# **Hyperparameter Tuning**
---

In [1]:
#Clear any logs from previous runs
!rm -rf ./logs/ 

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Not connected to a GPU


In [3]:
#Import of the relevant libaries
import numpy as np
import matplotlib.pyplot as plt 
import time
import pickle
import tensorflow as tf   
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import MobileNetV2, Xception, NASNetMobile, InceptionResNetV2
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam, Nadam
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from tensorboard.plugins.hparams import api as hp
import sklearn.metrics as metrics
#This is only necessary when you use google.colab and the trainingsdata are stored in google drive
from google.colab import drive
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import KFold

In [4]:
# Reduce learning rate when there is a change lesser than <min_delta> in <val_accuracy> for more than <patience> epochs
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_accuracy',
                                                 mode = 'max',
                                                 min_delta = 0.01,
                                                 patience = 3,
                                                 factor = 0.25,
                                                 verbose = 1,
                                                 cooldown = 0,
                                                 min_lr = 0.00000001)

# Stop the training process when there is a change lesser than <min_delta> in <val_accuracy> for more than <patience> epochs
early_stopper = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy',
                                                 mode = 'max',
                                                 min_delta = 0.005,
                                                 patience = 8,
                                                 verbose = 1,
                                                 restore_best_weights = True)

In [5]:
image_size = (224, 224)
batch_size = 32
epochs=50
num_classes = 4

## **Table of Conents**
---
[Own CNN Model](#cnn)<br>
[MobileNetV2](#mobile)<br>
[Xception](#xception)<br>
[NASNetMobile](#NASNetMobile)<br>
[ResNet](#ResNet)<br>
[Evaluation](#Evaluation)<br>

**Import the preprocessed data**

In [6]:
#This is only necessary when you use google.colab and the trainingsdata are stored in google drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#x_train = pickle.load(open("/content/drive/MyDrive/x_train2.pickle", "rb"))
#x_test = pickle.load(open("/content/drive/MyDrive/x_test2.pickle", "rb"))
#y_train = pickle.load(open("/content/drive/MyDrive/y_train2.pickle", "rb"))
#y_test = pickle.load(open("/content/drive/MyDrive/y_test2.pickle", "rb"))

# x_train = pickle.load(open("C:/Users/denni/OneDrive/Dokumente/Uni/Master/Masterarbeit/drive-download-20211109T191423Z-001/x_train.pickle", "rb"))
# x_test = pickle.load(open("C:/Users/denni/OneDrive/Dokumente/Uni/Master/Masterarbeit/drive-download-20211109T191423Z-001/x_test.pickle", "rb"))
# y_train = pickle.load(open("C:/Users/denni/OneDrive/Dokumente/Uni/Master/Masterarbeit/drive-download-20211109T191423Z-001/y_train.pickle", "rb"))
# y_test = pickle.load(open("C:/Users/denni/OneDrive/Dokumente/Uni/Master/Masterarbeit/drive-download-20211109T191423Z-001/y_test.pickle", "rb"))

In [ ]:
def random_contrast(x):
  return   tf.image.random_contrast(
      x, 0.1, 0.8
  )


In [21]:
datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        brightness_range = (0.4, 0.6),
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.2,
        preprocessing_function = tf.keras.applications.xception.preprocess_input
        )


train_generator = datagen.flow_from_directory(
        '/content/drive/MyDrive/Images/Train',
        target_size=image_size,
        color_mode='rgb',
        classes = ["Tel-Aviv","WestJerusalem" ,"Berlin", "Hamburg"],
        batch_size=batch_size,
        shuffle = True,
        seed = 777,
        #save_to_dir = '/content/fotos',
        #save_format='png', 
        follow_links=True,
        subset = 'training',
        class_mode='categorical')

validation_generator  = datagen.flow_from_directory(
        '/content/drive/MyDrive/Images/Train',
        target_size=image_size,
        color_mode='rgb',
        classes = ["Tel-Aviv","WestJerusalem" ,"Berlin", "Hamburg"],
        batch_size=batch_size,
        shuffle = True,
        seed = 777,
        #save_to_dir = '/content/fotos',
        #save_format='png', 
        follow_links=True,
        subset = 'validation',
        class_mode='categorical')

Found 9600 images belonging to 4 classes.
Found 2400 images belonging to 4 classes.


In [16]:
#HP_ACTIVATION = hp.HParam('activation', hp.Discrete(['selu']))
#HP_ACTIVATION = hp.HParam('activation', hp.Discrete(['relu', 'elu']))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))
#HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'adagrad','nadam']))
HP_FTA = hp.HParam('fta', hp.Discrete([100] ))
#HP_ALPHA= hp.HParam('alpha', hp.RealInterval(0.8  ,1.2 ))
#Settinf the Metric
METRIC = 'accuracy'


In [17]:
#Creating & configuring log files
with tf.summary.create_file_writer('/content/drive/MyDrive/logs').as_default():
    hp.hparams_config(
        hparams=[HP_OPTIMIZER, HP_FTA],
        metrics=[hp.Metric(METRIC, display_name='accuracy')]
    )

## Xception
<a id='xception'></a>

In [18]:
def create_model_x(hparams, input_shape, num_classes= 4 ):
    input_tensor = Input(shape=input_shape)
    base_model = Xception(
        include_top=False,
        weights='imagenet',
        input_tensor=input_tensor,
        input_shape=input_shape,
        pooling='avg')

    # Fine-tune from this layer onwards
    fine_tune_at = 100

    base_model.trainable = True

    # Freeze all the layers before the `fine_tune_at` layer
    for layer in base_model.layers[:hparams[HP_FTA]]:
        layer.trainable =  False
        
    #op = Dense(256, activation=hparams[HP_ACTIVATION])(base_model.output)
    op = Dropout(.25)(base_model.output)
    
    ##
    # sigmoid: calculates a probability for the class.
    ##
    output_tensor = Dense(num_classes, activation='softmax')(op)

    model = Model(inputs=input_tensor, outputs=output_tensor)
    
    model.compile(loss="categorical_crossentropy", optimizer=hparams[HP_OPTIMIZER], metrics=['accuracy'])
    model.fit(train_generator, batch_size=batch_size, epochs=epochs, use_multiprocessing=True, workers=8, validation_data = validation_generator , callbacks=[early_stopper, reduce_lr])
    test_loss, test_acc = model.evaluate(validation_generator, use_multiprocessing=True, workers=8, steps=train_generator.n // batch_size)

    return test_acc

In [19]:
def run_x(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  
        accuracy  = create_model_x(hparams=hparams, input_shape = image_size+(3,), num_classes=num_classes)
        tf.summary.scalar(METRIC, accuracy, step=10)

In [ ]:
#A unique number for each training session
session_num = 0

#Nested for loop training with all possible  combinathon of hyperparameters
for optimizer in HP_OPTIMIZER.domain.values:
    for fta in HP_FTA.domain.values:
      hparams = {
          HP_OPTIMIZER: optimizer,
          HP_FTA: fta
      }
      run_name = "run-%d" % session_num
      print('--- Starting trial: %s' % run_name)
      print({h.name: hparams[h] for h in hparams})
      run_x('/content/drive/MyDrive/logs/x/' + run_name  + ' ' + time.strftime('%X %x %Z') , hparams)
      session_num += 1

--- Starting trial: run-0
{'optimizer': 'adam', 'fta': 100}
Epoch 1/50
  4/300 [..............................] - ETA: 52:24 - loss: 1.3822 - accuracy: 0.2891

##Evaluation

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir drive/MyDrive/logs/x/

Launching TensorBoard...